In [ ]:
# 라이브러리 추가하기
import geopandas as gpd

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# GeoPandas로 지도시각화 하기

In [ ]:
# 서울시 시군구 지도 데이터 불러오기
seoul_sgg_pop = gpd.read_file('./maps/seoul_sgg_final.geojson')
seoul_sgg_pop.head()

In [ ]:
# 서울시 시군구 지도 데이터로 단계구분도(Chropleth Map) 시각화하기
fig, ax = plt.subplots(1, 1)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
seoul_sgg_pop.plot(column='TOTAL_POP', ax=ax, legend=True, cax=cax, cmap='OrRd')

In [ ]:
# 서울시 스타벅스 지도 데이터 불러오기
seoul_starbucks = gpd.read_file('./maps/seoul_starbucks.geojson')
seoul_starbucks.head()

In [ ]:
# plot
seoul_starbucks.plot()

In [ ]:
# 서울시 시군구 레이어 + 스타벅스 레이어
base_map = seoul_sgg_pop.plot(color='white', edgecolor='black')

In [ ]:
# seoul_starbucks.plot(ax=base_map, marker='o', color='red', markersize=5)
import platform
from matplotlib import font_manager, rc
if platform.system() == 'Windows':
    font_path = "c:/Windows/Fonts/malgun.ttf"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
# Mac OS
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')


%matplotlib inline

base_map = seoul_sgg_pop.plot(color='white', edgecolor="k")
ax = seoul_starbucks.plot(ax=base_map, marker='o', color='red', markersize=5)
ax.set_axis_off()
ax.set_title("서울시 스타벅스 분포도")
plt.show()

# Leaflet기반 지도 시각화하기

In [ ]:
# ipyleaflet 설치하기
# !pip install ipyleaflet
# !jupyter nbextension enable --py --sys-prefix ipyleaflet  # can be skipped for notebook 5.3 and above

In [ ]:
from ipyleaflet import Map, Marker, TileLayer, ImageOverlay, CircleMarker, GeoJSON, GeoData

In [ ]:
center = [37.573050, 126.979189]
m = Map(center=center, zoom=10)
m

In [ ]:
style = {'color': 'green', 'fill_color':'red', 'opacity':1, 'weight':1, 'dashArray':'9', 'fillOpacity':0.5}
geo_data = GeoData(geo_dataframe=seoul_sgg_pop, style=style)
m.add_layer(geo_data)
m
# marker = Marker(location=center, draggable=False)

In [ ]:
import json

seoul_sgg_geo = json.load(open('./maps/seoul_sgg_final.geojson', encoding='utf-8'))

for i in range(len(seoul_sgg_geo['features'])):
    seoul_sgg_geo['features'][i]['id'] = seoul_sgg_geo['features'][0]['properties']['SIG_CD']

In [ ]:
seoul_sgg_geo

In [ ]:
# 시군구별 단계구분도 작성하기
import ipyleaflet
from branca.colormap import linear

pop =  dict(zip(seoul_sgg_pop['SIG_CD'].tolist(), seoul_sgg_pop['TOTAL_POP'].tolist()))

m = Map(center=center, zoom=10)
layer = ipyleaflet.Choropleth(
    geo_data=seoul_sgg_geo,
    choro_data=pop,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})
m.add_layer(layer)
m

In [ ]:
# Circle Marker
m = Map(center=center, zoom=10)

style = {'color': 'black', 'opacity':1, 'weight':1, 'fillOpacity':0}
geo_data = GeoData(geo_dataframe=seoul_sgg_pop, style=style)
m.add_layer(geo_data)

centroid_list = seoul_sgg_pop['geometry'].centroid
total_pop = seoul_sgg_pop['TOTAL_POP']

for i in seoul_sgg_pop.index:
    lonlat = centroid_list[i]
    value = total_pop[i]
    
    circle_marker = CircleMarker()
    circle_marker.location = (lonlat.y, lonlat.x)
    circle_marker.radius = int(value / 100000)
    circle_marker.weight = 1
    circle_marker.fill_opacity=0.8
    circle_marker.color = "red"
    circle_marker.fill_color = "yellow"
    
    m.add_layer(circle_marker)

m